In [1]:
!pip install rouge

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import sys
import os
import re
import json 
import numpy as np
import pandas as pd
import nltk

In [3]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils
from lib.bleu_rouge import bleu_rouge

In [4]:
with open('../../output/prompt_ref.json') as f:
    prompt_ref = json.load(f)

In [5]:
def clean_text(text):
    text = re.sub(r'<[A-Z]+>|', '', text)
    text = re.sub(r' +', ' ', text).strip()
    text = re.sub(r'\n\n', '\n', text)
    text = re.sub(r'\n \n', '\n', text)
    text = re.sub(r' \n', '\n', text)
    text = re.sub(r'\n ', '\n', text)
    text = re.sub(r'\n+', '\n', text)
    text = text.split('\n')
    if len(text) > 1:
        prompt, generated = text[0], text[1]
    else:
        prompt, generated = text[0], text[-1]
    return prompt, generated

In [6]:
PATH = '../../output/'
all_files = os.listdir(PATH)
text_files = [file for file in all_files if re.search(r'\.txt$', file)]
br_files, human_files = [], []
for file in text_files:
    if 'br_' in file:
        br_files.append(file)
    elif 'human_' in file:
        human_files.append(file)

br_text = []
for file in br_files:
    with open(PATH + file) as f:
        file = file.replace('-', '_')
        text = clean_text(f.read())
        prompt = file.split('_')[-1].replace('.txt', '')
        if 'ae_' in file:            
            br_text.append(('_'.join(file.split('_')[1:-1]), prompt, text[-1].replace(prompt+' ', '')))
        else:
            br_text.append(('_'.join(file.split('_')[1:-1]), prompt, text[1]))

# human_text = []
# for file in human_files:
#     with open(PATH + file) as f:
#         text = clean_text(f.read())
#         human_text.append((file.split('_')[1], text[0], text[1]))

### Evaluate Bleu and Rouge Score

In [7]:
br_text[0]

('ae_lstm_att_mask', 'amazing in all ways', 'a new heart day that')

In [8]:
br = bleu_rouge(prompt_ref)

In [9]:
for i,x in enumerate(br_text):
    bleu = br.compute_bleu(x[1], x[2], verbose=False)['Avg']
    rouge = br.compute_rouge(x[1], x[2], verbose=False)
    rouge = (rouge['rouge-1']['r'] + rouge['rouge-2']['r'] + rouge['rouge-l']['r']) / 3
    br_text[i] = x + (bleu, rouge)

c:\Users\TeYang\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\TeYang\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\TeYang\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lowe

In [72]:
br_df = pd.DataFrame.from_records(br_text, columns=['model','prompt','generated', 'bleu', 'rouge'])
br_df = br_df.groupby(['model']).agg({'bleu':'mean', 'rouge':'mean'}).reset_index()
br_df = br_df.sort_values(['bleu', 'rouge'], ascending=False)
br_df

,model,bleu,rouge
0,GPT2,0.072678,0.074489
1,GRU,0.054326,0.074581
2,LSTM,0.053776,0.070360
3,RNN,0.034844,0.040848
8,ae_lstm_att_mask,0.022745,0.029675
5,ae_gru_att,0.005446,0.023640
6,ae_lstm,0.004096,0.010997
9,ae_rnn,0.002783,0.019138
10,ae_rnn_att,0.001207,0.015922
7,ae_lstm_att,0.000898,0.012788


In [10]:
br_df = pd.DataFrame.from_records(br_text, columns=['model','prompt','generated', 'bleu', 'rouge'])
br_df = br_df.groupby(['model']).agg({'bleu':'mean', 'rouge':'mean'}).reset_index()
br_df = br_df.sort_values(['bleu', 'rouge'], ascending=False)
br_df

,model,bleu,rouge
0,GPT2,0.072678,0.074489
1,GRU,0.054326,0.074581
2,LSTM,0.053776,0.070360
3,RNN,0.034844,0.040848
5,s2s_lstm_att,0.032500,0.039589
4,ae_lstm_att_mask,0.022745,0.029675


### Evaluate bleu and rouge for 6 standard prompts?

In [ ]:
standard_prompt_ref = {
    
}
br = bleu_rouge(prompt_ref)

In [16]:
for i,x in enumerate(human_text):
    bleu = br.compute_bleu(x[1], x[2], verbose=False)['Avg']
    rouge = br.compute_rouge(x[1], x[2], verbose=False)
    rouge = (rouge['rouge-1']['r'] + rouge['rouge-2']['r'] + rouge['rouge-l']['r']) / 3
    human_text[i] = x + (bleu, rouge)

AttributeError: PROMPT does NOT exist in sampled prompts.
            Run get_prompt_reference() to get prompt samples and check
            bleu_rogue.prompt_ref for the set of prompts and references